In [2]:
from openai import OpenAI
# import openai
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def get_chatbot_response(client, model_name, messages, temperature=0.0):
    input_messages = []
    for message in messages: 
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,  # no randomness desired as agent-based system will depend on each other
        top_p=0.8,
        max_tokens=2000  # word or sub-word
    ).choices[0].message.content

    return response

In [ ]:
# Connecting to Llama endpoint

client = OpenAI(
    api_key = os.getenv('RUNPOD_TOKEN'),
    base_url = os.getenv('RUNPOD_CHATBOT_URL')
)

# openai.api_key = os.getenv('RUNPOD_TOKEN')
# openai.api_base_url = os.getenv('RUNPOD_CHATBOT_URL')

model_name = os.getenv("MODEL_NAME")

print(model_name)
# print(client.api_key)

In [ ]:
messages = [
    {"role": "system", "content": "What is the capital of India?"}
]

response = get_chatbot_response(client, model_name, messages)

print(response)

In [19]:
# Test 

import requests
import os
from dotenv import load_dotenv

load_dotenv()

RUNPOD_TOKEN = os.getenv("RUNPOD_TOKEN")
RUNPOD_CHATBOT_URL = os.getenv("RUNPOD_CHATBOT_URL")  # e.g., https://<your-id>.proxy.runpod.net/v1

headers = {
    "Authorization": f"Bearer {RUNPOD_TOKEN}",
    "Content-Type": "application/json"
}

payload = {
    "model": "meta-llama/Llama-3.1-8B-Instruct",  
    "messages": [
        {"role": "user", "content": "What is the capital of India?"}
    ],
    "temperature": 0.7,
    "max_tokens": 200
}

response = requests.post(f"{RUNPOD_CHATBOT_URL}/chat/completions", headers=headers, json=payload)

print(response.status_code)
print(response.json())


403
{'error': 'You do not have permission to perform this action.'}


# Promt Engineering

## Structured output

In [ ]:
system_prompt = """
You are a helpful assistant that answers questions about the capitals of countries.

Your output should be a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:

{
    "country": the country that you will get the capital of,
    "capital": the capital of the country stated
}
"""

messages = [
    {"role": "system", "content": system_prompt},
]

messages.append({"role": "user", "content": "What is the capital of India?"})

response = get_chatbot_response(client, model_name, messages)

print(response)

In [ ]:
type(response)  # string

json_response = json.loads(response)

print(json_response)

type(json_response)  # list
type(json_response[0])  # dict

json_response[0]["country"]  # "India"
json_response[0]["capital"]  # "New Delhi"

## Input Structuring

In [ ]:
user_input = """ 
    Get me the capital of the following countries:
    ```
        1. Italy
        2. France
        3. Germany

    ```
"""

messages = [
    {"role": "system", "content": system_prompt},
]

messages.append({"role": "user", "content": user_input})

response = get_chatbot_response(client, model_name, messages)

print(response)  # list

In [ ]:
json_response = json.loads(response)

print(json_response)

## Give the model time to think (Chain of Thought)

In [ ]:
user_prompt = """
    Calculate the result of this equation: 1+3
    
    Your output should be a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:
    
    {
        "result": the final number resulted from calculating the equation above
    }
    
"""

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)

# But making the equation complex will cause the model to fail and give incorrect result
# We can make the accuracy better by telling it to do the calculation step by step

In [3]:
1234*43/567-123+94567

94537.58377425044

In [ ]:
user_prompt = """
    Calculate the result of this equation: 1234*43/567-123+94567
    
    Your output should be a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:
    
    {
        "result": the final number resulted from calculating the equation above
    }
    
"""

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)

In [ ]:
# Taking the code again

user_prompt = """
    Calculate the result of this equation: 1234*43/567-123+94567
    
    Your output should be a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:
    
    {
        "steps: This is where you solve the equation bit by bit following  the BEDMAS rule. You need to show your work and calculate each step leading to the final result. Feel free to write in free text",
        "result": the final number resulted from calculating the equation above
    }
    
"""

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)

# RAG - Retrieval Augmented Generation

In [ ]:
user_prompt = """
    What's new in iPhone 16?
"""

# Our LLM (Llama-3.1-8B-Instruct) does not know the existance of iPhone 16
# It was released after the model was trained

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)

In [ ]:
iPhone_16 = """
The iPhone 16 lineup brings notable upgrades: faster and smarter processing (A18 chip), expanded AI readiness, advanced camera features including a dedicated camera control button, improved battery and safety tools, and better customization with Photographic Styles. Whether you go for the standard or Pro version, Apple’s renewed focus on on-device intelligence and user control is its biggest leap since the iPhone X era.
"""

In [ ]:
user_prompt = f"""
    {iPhone_16}

    What's new in iPhone 16?
"""

messages = [{"role": "user", "content": user_prompt}]

response = get_chatbot_response(client, model_name, messages)

print(response)